In [1]:
""" This is Phase 1 project Movie Data Exploration File started on 6 May 2023"""
""" FIVE DATASETS: 
1. SQL - MOVIE DATA, im.db
2. CSV - movie grosses, bom.movie_gross.csv
3. TSV - movie info, rt.movie_info.tsv
4. CSV - MOVIE BUDGETS, movie_budgets.csv
5. TSV - reviews, rt.reviews.tsv
6. CSV - movies, movies.csv"""

import pandas as pd
import numpy as np
import sqlite3
import os
import zlib
import zipfile
from zipfile import ZipFile
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from pathlib import Path  
import glob


In [2]:
#This is a function to find the number of missing values per column in a dataframe
def missing_values(df_name):
    """This function iterates to find the number of missing values per column"""
    print ("Missing values by Column")
    for col in df_name.columns:
        missing = df_name[col].isna().sum()
        print (str(col) + ": " + str(missing) + " missing values")
    return

In [3]:
# This is a function to create a table of Value Counts for a particular column
def column_value_counts(df_name, col_name):
    list1 = []
    list_dict = {}
    for index, row in df_name.iterrows():
        value = row[col_name]
        if value in list1:
            list_dict[col_name] +=1
            continue
        else:
            list1.append(value)
            list_dict[col_name] = 1

    print("There are " + str(len(list1)) + " unique.")
    return

In [4]:
# 1) SQL - MOVIE DATA  

# Extract SQL db from zip
# ZipFile.extractall(file_path , members=None, pwd=None)Home\MovieData\


#textFile = zipfile.ZipFile("/home/hatenotwelcome/MovieData/im.db.zip", "r")
#textFile.close()

#SQL DB includes 8 tables: principals, known-for, directors, writers, persons, movie_basics, movie_ratings, movie_akas
# principals: *movie_id, 

conn = sqlite3.Connection("/home/bringingthesparkle/FlatIron/Movie_Data/im.db")

q = """
SELECT *
FROM principals
LIMIT 5
;
"""

pd.read_sql(q, conn)

# Connection closed at bottome of file

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"


In [5]:
# 2) MOVIE GROSSES

#This is the CSV containing  movie names and their domestic grosses and foreign grosses
# In this table, title has no missing values, but not sure if all unique, adding an index might be necessary
movie_grosses = pd.read_csv('/home/bringingthesparkle/FlatIron/Movie_Data/bom.movie_gross.csv.gz', sep=',')

print(movie_grosses.dtypes)
print(movie_grosses.tail(5))
print('\n')

missing_values(movie_grosses)


title              object
studio             object
domestic_gross    float64
foreign_gross      object
year                int64
dtype: object
                            title      studio  domestic_gross foreign_gross   
3382                    The Quake       Magn.          6200.0           NaN  \
3383  Edward II (2018 re-release)          FM          4800.0           NaN   
3384                     El Pacto        Sony          2500.0           NaN   
3385                     The Swan  Synergetic          2400.0           NaN   
3386            An Actor Prepares       Grav.          1700.0           NaN   

      year  
3382  2018  
3383  2018  
3384  2018  
3385  2018  
3386  2018  


Missing values by Column
title: 0 missing values
studio: 5 missing values
domestic_gross: 28 missing values
foreign_gross: 1350 missing values
year: 0 missing values


In [6]:
# creating new dataframe to represent only those with domestic gross, removing rows without
movie_grosses_domestic = movie_grosses.dropna(subset=['domestic_gross'])
# The initial shape had 3387 rows and 28 less is 3359
print (len(movie_grosses_domestic))
                                        
# then split/apply/combine on studio
studios = []
studios_dict = {}




column_value_counts(movie_grosses_domestic, 'studio')

#for index, row in movie_grosses_domestic.iterrows():
#    studio = row['studio']
#    if studio in studios:
#        studios_dict[studio] +=1
#        continue
#    else:
#        studios.append(studio)
#        studios_dict[studio] = 1

#print("There are " + str(len(studios)) + " unique studios.")
#print (studios)

df = pd.DataFrame(studios_dict.items(), columns=["Studio", "Studio_Count"])
print (df.sort_values(by=['Studio_Count'], ascending = False))



3359
There are 256 unique.
Empty DataFrame
Columns: [Studio, Studio_Count]
Index: []


In [7]:
# replace NaNS with Unknowns
movie_grosses = movie_grosses.replace(np.nan, 'Unknown')

for col in movie_grosses.columns:
    assert movie_grosses[col].isna().sum() == 0

print(movie_grosses.shape)

(3387, 5)


In [8]:
#3) CSV - MOVIE INFO

# This is the CSV with movie information with columns: id, synopsis, rating, genre, director, writer, theater_date, dvd_date, currency, box_office, runtime, studio.
# ID has type int64, all others are mixed type objects and may need cleaning. 
movie_info = pd.read_csv('/home/bringingthesparkle/FlatIron/Movie_Data/rt.movie_info.tsv', sep='\t')
#print(movie_info.head(2))

#print(movie_info.dtypes)
#print(movie_info.shape) # Shape is (1560 by 12)

# Assert that 'id' is a unique record and there are no missing values
assert movie_info["id"].isna().sum() == 0

#Using missing values function to count NaNs per column
missing_values(movie_info)


Missing values by Column
id: 0 missing values
synopsis: 62 missing values
rating: 3 missing values
genre: 8 missing values
director: 199 missing values
writer: 449 missing values
theater_date: 359 missing values
dvd_date: 359 missing values
currency: 1220 missing values
box_office: 1220 missing values
runtime: 30 missing values
studio: 1066 missing values


In [9]:
# Replacing NaNs and asserting that none remain

movie_info = movie_info.replace(np.nan, 'Unknown')

for col in movie_info.columns:
    assert movie_info[col].isna().sum() == 0

In [10]:
movie_info["genre"].value_counts().nlargest(10)
# Clearly the genre column contains more than one category and would be best if separated into dictionaries where you can query on the key

genre
Drama                                              151
Comedy                                             110
Comedy|Drama                                        80
Drama|Mystery and Suspense                          67
Art House and International|Drama                   62
Action and Adventure|Drama                          42
Action and Adventure|Drama|Mystery and Suspense     40
Drama|Romance                                       35
Comedy|Romance                                      32
Art House and International|Comedy|Drama            31
Name: count, dtype: int64

In [11]:
# Massage the data 

# Create a list of all genres to query on.
movie_genres = []
movie_genre_dict = {}

for index, row in movie_info.iterrows():
    massage = row['genre']
    data = massage.split("|")
    #print (data)
    for word in data:
        if word in movie_genres:
            movie_genre_dict[word] +=1
        else:
            movie_genres.append(word)
            movie_genre_dict[word] = 1

    

print("There are " + str(len(movie_genres)) + " unique genres.")
print (movie_genres)

df = pd.DataFrame(movie_genre_dict.items(), columns=["Genre", "Genre_Counts"])
print (df.sort_values(by=['Genre_Counts'], ascending = False))


There are 22 unique genres.
['Action and Adventure', 'Classics', 'Drama', 'Science Fiction and Fantasy', 'Musical and Performing Arts', 'Mystery and Suspense', 'Romance', 'Kids and Family', 'Comedy', 'Unknown', 'Documentary', 'Special Interest', 'Art House and International', 'Horror', 'Western', 'Television', 'Sports and Fitness', 'Animation', 'Faith and Spirituality', 'Cult Movies', 'Anime and Manga', 'Gay and Lesbian']
                          Genre  Genre_Counts
2                         Drama           912
8                        Comedy           550
0          Action and Adventure           366
5          Mystery and Suspense           309
12  Art House and International           265
6                       Romance           198
1                      Classics           193
3   Science Fiction and Fantasy           172
13                       Horror           134
7               Kids and Family            99
4   Musical and Performing Arts            98
10                  Do

In [12]:
# 4) CSV - MOVIE BUDGETS
#This is the file tn.movie_budgets.csv.gz. 

movie_budgets = pd.read_csv('/home/bringingthesparkle/FlatIron/Movie_Data/tn.movie_budgets.csv.gz', sep=',')

print(movie_budgets.dtypes)
print(movie_budgets.tail(5))
print('\n')

missing_values(movie_budgets)  #

id                    int64
release_date         object
movie                object
production_budget    object
domestic_gross       object
worldwide_gross      object
dtype: object
      id  release_date                          movie production_budget   
5777  78  Dec 31, 2018                         Red 11            $7,000  \
5778  79   Apr 2, 1999                      Following            $6,000   
5779  80  Jul 13, 2005  Return to the Land of Wonders            $5,000   
5780  81  Sep 29, 2015           A Plague So Pleasant            $1,400   
5781  82   Aug 5, 2005              My Date With Drew            $1,100   

     domestic_gross worldwide_gross  
5777             $0              $0  
5778        $48,482        $240,495  
5779         $1,338          $1,338  
5780             $0              $0  
5781       $181,041        $181,041  


Missing values by Column
id: 0 missing values
release_date: 0 missing values
movie: 0 missing values
production_budget: 0 missing values


In [47]:
# Exploration of Title Counts


from sklearn.feature_extraction.text import CountVectorizer

train_set = ["The sky is blue.", "The sun is bright."]
test_set = ["The sun in the sky is bright.", "We can see the shining sun, the bright sun."]
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(train_set) # I like data science and I'd like to really go back to grad school in eng, eng mgmt
print ("Vocabulary:")
print(count_vectorizer.vocabulary_)
Vocab = list(count_vectorizer.vocabulary_)
print(Vocab)

# Vocabulary: {'blue': 0, 'sun': 1, 'bright': 2, 'sky': 3}
freq_term_matrix = count_vectorizer.transform(test_set)
print (freq_term_matrix.todense())

count_array = freq_term_matrix.toarray()
df = pd.DataFrame(data=count_array, columns=Vocab)
print(df)

from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(freq_term_matrix)
print ("IDF:")
print(tfidf.idf_)



Vocabulary:
{'the': 5, 'sky': 3, 'is': 2, 'blue': 0, 'sun': 4, 'bright': 1}
['the', 'sky', 'is', 'blue', 'sun', 'bright']
[[0 1 1 1 1 2]
 [0 1 0 0 2 2]]
   the  sky  is  blue  sun  bright
0    0    1   1     1    1       2
1    0    1   0     0    2       2
IDF:
[2.09861229 1.         1.40546511 1.40546511 1.         1.        ]


In [ ]:
#5) TSV - reviews, rt.reviews.tsv
# ratings is in multiple different forms and would need manipulation

movie_info = pd.read_csv('/home/bringingthesparkle/FlatIron/Movie_Data/rt.reviews.tsv', sep='\t', encoding='latin1')
print(movie_info.head(2))

print(movie_info.dtypes)
print(movie_info.shape) # Shape is (1560 by 12)

# Assert that 'id' is a unique record and there are no missing values
assert movie_info["id"].isna().sum() == 0

#Using missing values function to count NaNs per column
missing_values(movie_info)

   id                                             review rating   fresh   
0   3  A distinctly gallows take on contemporary fina...    3/5   fresh  \
1   3  It's an allegory in search of a meaning that n...    NaN  rotten   

           critic  top_critic        publisher               date  
0      PJ Nabarro           0  Patrick Nabarro  November 10, 2018  
1  Annalee Newitz           0          io9.com       May 23, 2018  
id             int64
review        object
rating        object
fresh         object
critic        object
top_critic     int64
publisher     object
date          object
dtype: object
(54432, 8)
Missing values by Column
id: 0 missing values
review: 5563 missing values
rating: 13517 missing values
fresh: 0 missing values
critic: 2722 missing values
top_critic: 0 missing values
publisher: 309 missing values
date: 0 missing values


In [ ]:
#6) CSV - movies, tmdb_movies.csv

movie_budgets = pd.read_csv('/home/bringingthesparkle/FlatIron/Movie_Data/tmdb.movies.csv', sep=',')

print(movie_budgets.dtypes)
print(movie_budgets.tail(5))
print('\n')

# Assert that 'id' is a unique record and there are no missing values
assert movie_info["id"].isna().sum() == 0
 #


Unnamed: 0             int64
genre_ids             object
id                     int64
original_language     object
original_title        object
popularity           float64
release_date          object
title                 object
vote_average         float64
vote_count             int64
dtype: object
       Unnamed: 0        genre_ids      id original_language   
26512       26512         [27, 18]  488143                en  \
26513       26513         [18, 53]  485975                en   
26514       26514     [14, 28, 12]  381231                en   
26515       26515  [10751, 12, 28]  366854                en   
26516       26516         [53, 27]  309885                en   

              original_title  popularity release_date                  title   
26512  Laboratory Conditions         0.6   2018-10-13  Laboratory Conditions  \
26513        _EXHIBIT_84xxx_         0.6   2018-05-01        _EXHIBIT_84xxx_   
26514           The Last One         0.6   2018-10-01           The Las

In [ ]:
# check for missing values
missing_values(movie_budgets) 

Missing values by Column
Unnamed: 0: 0 missing values
genre_ids: 0 missing values
id: 0 missing values
original_language: 0 missing values
original_title: 0 missing values
popularity: 0 missing values
release_date: 0 missing values
title: 0 missing values
vote_average: 0 missing values
vote_count: 0 missing values


In [ ]:

# Gotta close that SQL file connection from earlier
conn.close()